In [1]:
import cv2
import numpy as np
# from tensorflow import keras as k

In [2]:
path = r'C:\\Users\\Administrator\\Desktop\elephant\\OCR_test\\20190309162316.png'

def cv_show(img, name):
    cv2.imshow(name, img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [4]:
img = cv2.imread(path)
cv_show(img, 'img')

In [5]:
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# print(imgGray.shape)
imgGray_r = cv2.resize(imgGray,(342,514))    #干，shape是按高宽显示，resize是按宽高显示，正好相反，太久没学习忘掉了........
# print(imgGray_r.shape)
# cv_show(imgGray_r,'imgGray_r')

In [6]:
kernel = np.ones((5,5),np.uint8)

imgcropped = imgGray_r[152:495, 67:253]    #先高后宽：从上到下，从左到右
# cv2.putText(imgcropped,"cute lady",(50,150),cv2.FONT_ITALIC,3,(255,0,255,1))
# imgcanny = cv2.Canny(imgcropped,150,200)

# imgEroded = cv2.erode(imgDilation,kernel,iterations=1)
# imgDilation = cv2.dilate(imgcropped,kernel,iterations=1)

# cv_show(imgcropped,'imgcropped')
# cv_show(imgcropped,'imgcropped')

In [7]:
# 以下利用高斯金字塔上采样&下采样


up = cv2.pyrUp(imgGray_r)    #上采样：将图像在每个方向扩大为原来的两倍，新增的行列以0填充
# cv_show(up,'pyrUP')

In [8]:
down = cv2.pyrDown(imgGray_r)    #降采样
# cv_show(down,'pyrDown')

In [9]:
up_down = cv2.pyrUp(down)
# cv_show(up_down,'pyrup_down')

In [10]:
cv_show(np.hstack((imgGray_r, up_down,imgGray_r-up_down)),'compare')    #hstack是并排放置，vstack是纵排放置
#imgGray_r-up_down  这里进行了一次拉普拉斯金字塔采样

In [ ]:
cv2.findContours()